# <font size=5> <strong>Heart Disease Prediction
 

## 1. Importing Libraries

In [1]:
# Core libraries
import numpy as np
import pandas as pd
import torch

# Data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn.metrics import(accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,
                            auc, confusion_matrix, classification_report, make_scorer)

# Modeling – XGBoost and TabNet
import xgboost as xgb
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier

# Hyperparameter optimization
import optuna
from sklearn.model_selection import GridSearchCV

# Explainability
import shap
import random
import os

# Visualization
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall
import keras_tuner as kt
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import backend as K

In [2]:
import numpy as np
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## 2. Data Preprocessing

### Importing datasets

In [71]:
df_A = pd.read_csv("Cleveland+Hungary+VA_long_beach+Switzerland.csv") # Source domain = multi-hospital dataset
df_B = pd.read_csv("Heart_disease_cleveland.csv")                     # Target domain = original Cleveland dataset

### Exploring and Inspecting Data sets

In [4]:
df_A.info() # Displays concise summary of DataFrame A: index range, column names, non-null counts, and data types.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        919 non-null    int64  
 1   age       919 non-null    int64  
 2   sex       919 non-null    object 
 3   dataset   919 non-null    object 
 4   cp        919 non-null    object 
 5   trestbps  919 non-null    float64
 6   chol      919 non-null    float64
 7   fbs       919 non-null    bool   
 8   restecg   919 non-null    object 
 9   thalch    919 non-null    float64
 10  exang     919 non-null    bool   
 11  oldpeak   919 non-null    float64
 12  slope     919 non-null    object 
 13  ca        919 non-null    float64
 14  thal      919 non-null    object 
 15  num       919 non-null    int64  
dtypes: bool(2), float64(5), int64(3), object(6)
memory usage: 102.4+ KB


In [5]:
df_B.info() # Displays concise summary of DataFrame B: index range, column names, non-null counts, and data types.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
# Remove Cleveland data from df_A before pretraining
df_A = df_A[df_A['dataset'] != 'Cleveland']

In [7]:
print(df_A.columns) # Prints the list of column names in DataFrame A.
print(df_B.columns) # Prints the list of column names in DataFrame B.

Index(['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs',
       'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')
Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [8]:
df_A = df_A.rename(columns={'thalch': 'thalach','num': 'target'}) #Rename columns in Dataset A to match Dataset B.

In [9]:
df_A = df_A.drop(columns=['id', 'dataset']) #Drop irrelevant columns in Dataset A (id, dataset).

In [10]:
# Convert target column in Dataset A to binary (0=no disease, 1=disease)
df_A['target'] = df_A['target'].apply(lambda x: 1 if x > 0 else 0)

In [11]:
# Map all categorical strings/booleans in Dataset A to numeric codes matching the target domain.
sex_map = {'Male': 1, 'Female': 0}

cp_map = {
    'typical angina': 0,
    'atypical angina': 1,
    'non-anginal': 2,
    'asymptomatic': 3
}

fbs_map = {True: 1, False: 0}

restecg_map = {
    'normal': 0,
    'st-t abnormality': 1,
    'lv hypertrophy': 2
}

exang_map = {True: 1, False: 0}

slope_map = {
    'upsloping': 0,
    'flat': 1,
    'downsloping': 2
}

thal_map = {
    'normal': 1,
    'fixed defect': 2,
    'reversable defect': 3
}

In [12]:
# Align features so both datasets have identical column names and encodings.
df_A['sex'] = df_A['sex'].map(sex_map)
df_A['cp'] = df_A['cp'].map(cp_map)
df_A['fbs'] = df_A['fbs'].map(fbs_map)
df_A['restecg'] = df_A['restecg'].map(restecg_map)
df_A['exang'] = df_A['exang'].map(exang_map)
df_A['slope'] = df_A['slope'].map(slope_map)
df_A['thal'] = df_A['thal'].map(thal_map)

In [13]:
# Final Check (Both should return zero missing values.)
print(df_A.isnull().sum())
print(df_B.isnull().sum())

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [14]:
#Do a final check for the target domain to ensure all columns are non-null.
df_A.info()

<class 'pandas.core.frame.DataFrame'>
Index: 615 entries, 304 to 918
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       615 non-null    int64  
 1   sex       615 non-null    int64  
 2   cp        615 non-null    int64  
 3   trestbps  615 non-null    float64
 4   chol      615 non-null    float64
 5   fbs       615 non-null    int64  
 6   restecg   615 non-null    int64  
 7   thalach   615 non-null    float64
 8   exang     615 non-null    int64  
 9   oldpeak   615 non-null    float64
 10  slope     615 non-null    int64  
 11  ca        615 non-null    float64
 12  thal      615 non-null    int64  
 13  target    615 non-null    int64  
dtypes: float64(5), int64(9)
memory usage: 72.1 KB


In [15]:
#Do a final check for the source domain to ensure all columns are non-null. 
df_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [16]:
BOLD = '\033[1m'
END = '\033[0m'

print(f"{BOLD}Datasets after preprocessing:{END}")

print(f"{BOLD}Multi-hospital Dataset:{END}")
print(f"{BOLD}{df_A.head(3).T.to_string(line_width=1000)}{END}")

print(f"\n{BOLD}Cleveland Dataset:{END}")
print(f"{BOLD}{df_B.head(3).T.to_string(line_width=1000)}{END}")

Datasets after preprocessing:
Multi-hospital Dataset:
            304     305    306
age        29.0   29.00   30.0
sex         1.0    1.00    0.0
cp          1.0    1.00    0.0
trestbps  120.0  140.00  170.0
chol      243.0  240.48  237.0
fbs         0.0    0.00    0.0
restecg     0.0    0.00    1.0
thalach   160.0  170.00  170.0
exang       0.0    0.00    0.0
oldpeak     0.0    0.00    0.0
slope       0.0    0.00    0.0
ca          0.0    0.00    0.0
thal        1.0    1.00    2.0
target      0.0    0.00    0.0

Cleveland Dataset:
              0      1      2
age        63.0   67.0   67.0
sex         1.0    1.0    1.0
cp          0.0    3.0    3.0
trestbps  145.0  160.0  120.0
chol      233.0  286.0  229.0
fbs         1.0    0.0    0.0
restecg     2.0    2.0    2.0
thalach   150.0  108.0  129.0
exang       0.0    1.0    1.0
oldpeak     2.3    1.5    2.6
slope       2.0    1.0    1.0
ca          0.0    3.0    2.0
thal        2.0    1.0    3.0
target      0.0    1.0    1.0


## 3. Data split

In [17]:
X_pretrain = df_A.drop('target', axis=1) 
y_pretrain = df_A['target']               

# Split the source domain into 80% train and 20% temp (for val+test)
X_pretrain_train, X_pretrain_temp, y_pretrain_train, y_pretrain_temp = train_test_split(
    X_pretrain, y_pretrain,
    test_size=0.2,          # 20% goes to temp (val+test)
    random_state=42,
    stratify=y_pretrain
)

# Split temp into 50% validation and 50% test → each 10% of total
X_pretrain_val, X_pretrain_test, y_pretrain_val, y_pretrain_test = train_test_split(
    X_pretrain_temp, y_pretrain_temp,
    test_size=0.5,          # half of temp = 10% of total
    random_state=42,
    stratify=y_pretrain_temp
)

# Target domain: Cleveland dataset for FINE-TUNING and evaluation
X_finetune = df_B.drop('target', axis=1)
y_finetune = df_B['target']


# Split target domain into 80% train and 20% temp (for val+test)
X_finetune_train, X_finetune_temp, y_finetune_train, y_finetune_temp = train_test_split(
    X_finetune, y_finetune,
    test_size=0.2,          # 20% goes to temp (val+test) 
    random_state=42,        # reproducible splits
    stratify=y_finetune     # preserve class distribution
)

# Split temp into 50% validation and 50% test → each 10% of total
X_finetune_val, X_finetune_test, y_finetune_val, y_finetune_test = train_test_split(
    X_finetune_temp, y_finetune_temp,
    test_size=0.5,          # 20% of training data will be used as the validation set
    random_state=42,        # reproducible splits
    stratify=y_finetune_temp  # preserve class distribution
)

## 4. Class imbalance handling using SMOTE

In [23]:
# Apply SMOTE to source domain data (X_pretrain_train, y_pretrain_train)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_pretrain_train_res, y_pretrain_train_res = smote.fit_resample(X_pretrain_train, y_pretrain_train)

# Apply SMOTE to target domain data (X_finetune_train, y_finetune_train)
X_finetune_train_res, y_finetune_train_res = smote.fit_resample(X_finetune_train, y_finetune_train)

# Check the resampled data shapes and class distributions
print(f"Resampled Dataset A training data shape: {X_pretrain_train_res.shape}")
print(f"Resampled Dataset B training data shape: {X_finetune_train_res.shape}")

# Optionally, you can print the class distribution before and after SMOTE
print(f"Class distribution before SMOTE (Source domain): \n{y_pretrain_train.value_counts()}")
print(f"Class distribution after SMOTE (Source domain): \n{y_pretrain_train_res.value_counts()}")

print(f"Class distribution before SMOTE (Target domain): \n{y_finetune_train.value_counts()}")
print(f"Class distribution after SMOTE (Target domain): \n{y_finetune_train_res.value_counts()}")


Resampled Dataset A training data shape: (590, 10)
Resampled Dataset B training data shape: (262, 10)
Class distribution before SMOTE (Source domain): 
target
1    295
0    197
Name: count, dtype: int64
Class distribution after SMOTE (Source domain): 
target
1    295
0    295
Name: count, dtype: int64
Class distribution before SMOTE (Target domain): 
target
0    131
1    111
Name: count, dtype: int64
Class distribution after SMOTE (Target domain): 
target
1    131
0    131
Name: count, dtype: int64


## 5. XGBoost

### Pretraining on the source domain

#### Hyperparameter tuning for Pre-training

In [23]:
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0.5, 1, 1.5],
}

# Create XGBoost with balanced class weight
xgb_pretrain_base = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    early_stopping_rounds=50,
    n_jobs=1,                 # single-threaded = deterministic
    scale_pos_weight=1
)

grid_search = GridSearchCV(
    estimator=xgb_pretrain_base,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=1,                 # ALSO single-threaded
    verbose=2
)

grid_search.fit(
    X_pretrain_train_res,
    y_pretrain_train_res,
    eval_set=[(X_pretrain_val, y_pretrain_val)],
    verbose=False
)

# Use best estimator 
xgb_pretrain_best = grid_search.best_estimator_

Fitting 5 folds for each of 13824 candidates, totalling 69120 fits


In [24]:
print(f"\nBest parameters for pretraining: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_:.4f}")

# UPDATED
# Best parameters for pretraining: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100,
#                                   'reg_alpha': 0, 'reg_lambda': 0.5, 'subsample': 0.8}
# Best F1-score: 0.8994


Best parameters for pretraining: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 200, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 0.7}
Best F1-score: 0.8965


#### Pre-training on the source domain

In [24]:
# Define the best parameters found during the Grid Search for pretraining on Dataset A......DO NOT RUN THIS CODE IF YOU DO HYPERPARAMETER TUNING
best_pretrain_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.01,
    'max_depth': 8,
    'n_estimators': 300,
    'reg_alpha': 0.1, 
    'reg_lambda': 0.5, 
    'subsample': 1.0
}

In [25]:
# Get the best parameters....UNCOMMENT THE LINE BELOW IF YOU DO HYPERPARAMETER TUNING
# best_pretrain_params = grid_search.best_params_

for param, value in best_pretrain_params.items():
    print(f"  {param}: {value}")

# Create and train fresh model
xgb_pretrain = xgb.XGBClassifier(
    **best_pretrain_params,
    eval_metric='logloss',
    random_state=42,   
    early_stopping_rounds=50,
    n_jobs=1             
)

xgb_pretrain.fit(
    X_pretrain_train_res,
    y_pretrain_train_res,
    eval_set=[(X_pretrain_val, y_pretrain_val)],
    verbose=1
)

  colsample_bytree: 0.7
  learning_rate: 0.01
  max_depth: 8
  n_estimators: 300
  reg_alpha: 0.1
  reg_lambda: 0.5
  subsample: 1.0
[0]	validation_0-logloss:0.68962
[1]	validation_0-logloss:0.68509
[2]	validation_0-logloss:0.68114
[3]	validation_0-logloss:0.67682
[4]	validation_0-logloss:0.67239
[5]	validation_0-logloss:0.66902
[6]	validation_0-logloss:0.66489
[7]	validation_0-logloss:0.66074
[8]	validation_0-logloss:0.65727
[9]	validation_0-logloss:0.65349
[10]	validation_0-logloss:0.64910
[11]	validation_0-logloss:0.64457
[12]	validation_0-logloss:0.64099
[13]	validation_0-logloss:0.63783
[14]	validation_0-logloss:0.63401
[15]	validation_0-logloss:0.63069
[16]	validation_0-logloss:0.62735
[17]	validation_0-logloss:0.62396
[18]	validation_0-logloss:0.62049
[19]	validation_0-logloss:0.61672
[20]	validation_0-logloss:0.61361
[21]	validation_0-logloss:0.61123
[22]	validation_0-logloss:0.60757
[23]	validation_0-logloss:0.60468
[24]	validation_0-logloss:0.60166
[25]	validation_0-logloss:0

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=1,
              num_parallel_tree=None, ...)

In [26]:
# PRETRAINED MODEL EVALUATION ON A-TEST
Dataset_A_Pretrain = xgb_pretrain.predict(X_pretrain_test)

#Calculate and print metrics on source domain's test set
Dataset_A_Pretrain_acc = accuracy_score(y_pretrain_test, Dataset_A_Pretrain) * 100
Dataset_A_Pretrain_f1  = f1_score(y_pretrain_test, Dataset_A_Pretrain) * 100

cm_A = confusion_matrix(y_pretrain_test, Dataset_A_Pretrain)
TN_A, FP_A, FN_A, TP_A = cm_A.ravel()


recall_A = TP_A / (TP_A + FN_A) * 100
specificity_A = TN_A / (TN_A + FP_A) * 100
print(f"Pretraining Test Accuracy on Source domain: {Dataset_A_Pretrain_acc:.2f}%")
print(f"Pretraining Test F1 on Source domain: {Dataset_A_Pretrain_f1:.2f}%")
print(f"Pretraining Test Recall (Sensitivity) on Source domain: {recall_A:.2f}%")
print(f"Pretraining Test Specificity on Source domain: {specificity_A:.2f}%")

Pretraining Test Accuracy on Source domain: 85.48%
Pretraining Test F1 on Source domain: 88.31%
Pretraining Test Recall (Sensitivity) on Source domain: 91.89%
Pretraining Test Specificity on Source domain: 76.00%


In [27]:
Dataset_B_Pretrain = xgb_pretrain.predict(X_finetune_test)

#Calculate and print metrics on target domain's test set
Dataset_B_Pretrain_acc = accuracy_score(y_finetune_test, Dataset_B_Pretrain) * 100
Dataset_B_Pretrain_f1 = f1_score(y_finetune_test, Dataset_B_Pretrain) * 100
cm_B = confusion_matrix(y_finetune_test, Dataset_B_Pretrain)
TN_B, FP_B, FN_B, TP_B = cm_B.ravel()
recall_B = TP_B / (TP_B + FN_B) * 100
specificity_B = TN_B / (TN_B + FP_B) * 100

print(f"Pretraining Accuracy on target domain's test Set: {Dataset_B_Pretrain_acc:.2f}%")
print(f"Pretraining F1 on target domain's test Set: {Dataset_B_Pretrain_f1:.2f}%")
print(f"Pretraining Recall (Sensitivity) on target domain's test Set: {recall_B:.2f}%")
print(f"Pretraining Specificity on target domain's test Set: {specificity_B:.2f}%")

Pretraining Accuracy on target domain's test Set: 74.19%
Pretraining F1 on target domain's test Set: 75.00%
Pretraining Recall (Sensitivity) on target domain's test Set: 85.71%
Pretraining Specificity on target domain's test Set: 64.71%


### Finetuning on the target domain

#### Hyperparameter tuning for Fine-tuning

In [28]:
param_grid_finetune = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.001, 0.01, 0.05],
    'n_estimators': [100, 200, 300],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0.5, 1, 1.5],
}

xgb_finetune_best = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    early_stopping_rounds=50,
    n_jobs=1,
    scale_pos_weight=1
)

grid_search_finetune = GridSearchCV(
    estimator=xgb_finetune_best,
    param_grid=param_grid_finetune,
    scoring='f1',
    cv=5,
    verbose=2,
    n_jobs=1
)

grid_search_finetune.fit(
    X_finetune_train_res,
    y_finetune_train_res,
    eval_set=[(X_finetune_val, y_finetune_val)],
    xgb_model=xgb_pretrain.get_booster(),
    verbose=False
)

xgb_finetune_best = grid_search_finetune.best_estimator_

Fitting 5 folds for each of 2916 candidates, totalling 14580 fits


In [29]:
print(f"\nBest parameters for finetuning: {grid_search_finetune.best_params_}")
print(f"Best F1-score: {grid_search_finetune.best_score_:.4f}")

# Best parameters for finetuning: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 4, 
#                                  'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 0.5, 'subsample': 0.7}
# Best F1-score: 0.8231


Best parameters for finetuning: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 0.5, 'subsample': 0.7}
Best F1-score: 0.8216


#### Finetuning on the target domain

In [28]:
# #DO NOT THE LINE CODE BELOW IF YOU DO HYPERPARAMETER TUNING
best_finetune_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.009,
    'max_depth': 6,
    'n_estimators': 300,
    'reg_alpha': 0.1, 
    'reg_lambda': 1.5, 
    'subsample': 1.0
}

In [29]:
# Get the best parameters from fine-tuning grid search......UNCOMMENT THE LINE BELOW IF YOU DO HYPERPARAMETER TUNING
# best_finetune_params = grid_search_finetune.best_params_

for param, value in best_finetune_params.items():
    print(f"  {param}: {value}")

xgb_finetune = xgb.XGBClassifier(
    **best_finetune_params,
    eval_metric='logloss',
    random_state=42,
    early_stopping_rounds=50,
    n_jobs=1
)

xgb_finetune.fit(
    X_finetune_train_res,
    y_finetune_train_res, 
    eval_set=[(X_finetune_val, y_finetune_val)], 
    xgb_model=xgb_pretrain.get_booster(),
    verbose=1 
)

  colsample_bytree: 0.7
  learning_rate: 0.009
  max_depth: 6
  n_estimators: 300
  reg_alpha: 0.1
  reg_lambda: 1.5
  subsample: 1.0
[0]	validation_0-logloss:0.31364
[1]	validation_0-logloss:0.31171
[2]	validation_0-logloss:0.31046
[3]	validation_0-logloss:0.30874
[4]	validation_0-logloss:0.30726
[5]	validation_0-logloss:0.30543
[6]	validation_0-logloss:0.30367
[7]	validation_0-logloss:0.30296
[8]	validation_0-logloss:0.30126
[9]	validation_0-logloss:0.30033
[10]	validation_0-logloss:0.29865
[11]	validation_0-logloss:0.29749
[12]	validation_0-logloss:0.29669
[13]	validation_0-logloss:0.29564
[14]	validation_0-logloss:0.29440
[15]	validation_0-logloss:0.29289
[16]	validation_0-logloss:0.29135
[17]	validation_0-logloss:0.29038
[18]	validation_0-logloss:0.28935
[19]	validation_0-logloss:0.28867
[20]	validation_0-logloss:0.28775
[21]	validation_0-logloss:0.28648
[22]	validation_0-logloss:0.28564
[23]	validation_0-logloss:0.28466
[24]	validation_0-logloss:0.28360
[25]	validation_0-logloss:

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.009, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=1,
              num_parallel_tree=None, ...)

In [30]:
# FINE-TUNED MODEL EVALUATION ON A-TEST (Checking for Catastrophic Forgetting)
Dataset_A_Finetune = xgb_finetune.predict(X_pretrain_test) # Using the fine-tuned model on A's test set

# Calculate and print metrics on source domain's test set
acc_A_Finetune = accuracy_score(y_pretrain_test, Dataset_A_Finetune) * 100
f1_A_Finetune = f1_score(y_pretrain_test, Dataset_A_Finetune) * 100

cm_A_Finetune = confusion_matrix(y_pretrain_test, Dataset_A_Finetune)
TN_A_F, FP_A_F, FN_A_F, TP_A_F = cm_A_Finetune.ravel()

recall_A_Finetune = TP_A_F / (TP_A_F + FN_A_F) * 100
specificity_A_Finetune = TN_A_F / (TN_A_F + FP_A_F) * 100

print(f"Fine-Tuning Test Accuracy on Source domain: {acc_A_Finetune:.2f}%")
print(f"Fine-Tuning Test F1 on Source domain: {f1_A_Finetune:.2f}%")
print(f"Fine-Tuning Test Recall (Sensitivity) on Source domain: {recall_A_Finetune:.2f}%")
print(f"Fine-Tuning Test Specificity on Source domain: {specificity_A_Finetune:.2f}%")

Fine-Tuning Test Accuracy on Source domain: 80.65%
Fine-Tuning Test F1 on Source domain: 84.62%
Fine-Tuning Test Recall (Sensitivity) on Source domain: 89.19%
Fine-Tuning Test Specificity on Source domain: 68.00%


In [31]:
# FINE-TUNED MODEL EVALUATION ON B-TEST (Final Transfer Learning Performance)
Dataset_B_Finetune = xgb_finetune.predict(X_finetune_test) # Using the fine-tuned model on B's test set

# Calculate and print metrics on target domain's test set
acc_B_Finetune = accuracy_score(y_finetune_test, Dataset_B_Finetune) * 100
f1_B_Finetune = f1_score(y_finetune_test, Dataset_B_Finetune) * 100

cm_B_Finetune = confusion_matrix(y_finetune_test, Dataset_B_Finetune)
TN_B_F, FP_B_F, FN_B_F, TP_B_F = cm_B_Finetune.ravel()

recall_B_Finetune = TP_B_F / (TP_B_F + FN_B_F) * 100
specificity_B_Finetune = TN_B_F / (TN_B_F + FP_B_F) * 100
roc_auc_B_Finetune = roc_auc_score(y_finetune_test, Dataset_B_Finetune) * 100

print(f"Fine-Tuning Test ROC-AUC on Target domain (B): {roc_auc_B_Finetune:.2f}%")
print(f"Fine-Tuning Test Accuracy on Target domain (B): {acc_B_Finetune:.2f}%")
print(f"Fine-Tuning Test F1 on Target domain (B): {f1_B_Finetune:.2f}%")
print(f"Fine-Tuning Test Recall (Sensitivity) on Target domain (B): {recall_B_Finetune:.2f}%")
print(f"Fine-Tuning Test Specificity on Target domain (B): {specificity_B_Finetune:.2f}%")

Fine-Tuning Test ROC-AUC on Target domain (B): 85.29%
Fine-Tuning Test Accuracy on Target domain (B): 83.87%
Fine-Tuning Test F1 on Target domain (B): 84.85%
Fine-Tuning Test Recall (Sensitivity) on Target domain (B): 100.00%
Fine-Tuning Test Specificity on Target domain (B): 70.59%


In [38]:
# Set global font to Times New Roman
# plt.rcParams.update({'font.family': 'Times New Roman'})
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size' : '16'})

# Assuming y_finetune_test and Dataset_B_Finetune are already defined
cm = confusion_matrix(y_finetune_test, Dataset_B_Finetune)

# Unravel the confusion matrix to get TN, FP, FN, TP
tn, fp, fn, tp = cm.ravel()

plt.figure(figsize=(5, 4))
im = plt.imshow(cm, cmap='Blues', interpolation='nearest') 

vmax = cm.max()
colorbar_ticks = np.arange(0, vmax + 10, 10)
plt.colorbar(im, ticks=colorbar_ticks)

# plt.title('Confusion Matrix of the fine-tuned model')

ax = plt.gca()
# ax.invert_yaxis()  # You can uncomment this if you want to invert the Y-axis

# Set X-axis labels (Predicted: Disease then No Disease)
plt.xticks([0, 1], ['No Disease', 'Disease']) 
plt.yticks([0, 1], ['No Disease', 'Disease']) 

plt.ylabel('Actual')
plt.xlabel('Predicted')

# Dynamic text color for contrast
norm = mcolors.Normalize(vmin=cm.min(), vmax=cm.max())
cmap = plt.colormaps['Blues']

for i in range(2):
    for j in range(2):
        cell_value = cm[i, j]
        cell_color = cmap(norm(cell_value))
        luminance = 0.299 * cell_color[0] + 0.587 * cell_color[1] + 0.114 * cell_color[2]
        text_color = 'white' if luminance < 0.5 else 'black'
        
        # Set the text in Times New Roman
        plt.text(j, i, f'{cell_value}', ha='center', va='center', color=text_color, fontsize=24, fontweight='bold')

# Save the confusion matrix image
plt.savefig('confusion_matrix_xgboost.png', dpi=300, bbox_inches='tight')
plt.close()


In [39]:
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size' : '16'})

# Calculate Precision, Recall, F1-Score
precision = precision_score(y_finetune_test, Dataset_B_Finetune) * 100
recall = recall_score(y_finetune_test, Dataset_B_Finetune) * 100
f1 = f1_score(y_finetune_test, Dataset_B_Finetune) * 100

# Calculate Specificity: TN / (TN + FP)
specificity = (tn / (tn + fp)) * 100

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_finetune_test, Dataset_B_Finetune) * 100

print(f"ROC-AUC: {roc_auc:.2f}%")

# Plot ROC curve with blue tones (IEEE standard)
fpr, tpr, _ = roc_curve(y_finetune_test, Dataset_B_Finetune)
roc_auc_value = auc(fpr, tpr)

plt.figure(figsize=(7, 6))

# Plot the ROC curve: Use 'orange' color and update the label
plt.plot(fpr, tpr, color='orange', lw=2, label=f'ROC curve (AUC = {roc_auc_value:.2f})')
plt.plot([0, 1], [0, 1], color='darkblue', lw=2, linestyle='--')

# Set limits and labels as in the original image
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate') # Changed back to match the image axis label

# Adjust legend location to match the original image
plt.legend(loc='lower right')

# Save the ROC curve image (you can keep your original filename or update it)
plt.savefig('roc_curve_xgboost.png', dpi=300, bbox_inches='tight')
plt.close()

ROC-AUC: 82.35%


### XGBoost with no Transfer Learning

In [34]:
baseline_model = XGBClassifier(
    n_estimators=300,            
    max_depth=6,
    learning_rate=0.009,
    subsample=1.0,
    colsample_bytree=0.7,
    eval_metric='logloss',
    reg_lambda = 1.5,
    reg_alpha = 0.1,
    early_stopping_rounds=50,
    random_state=42,
    n_jobs=1
)

baseline_model.fit(
    X_finetune_train_res,
    y_finetune_train_res,
    eval_set=[(X_finetune_val, y_finetune_val)]
)

[0]	validation_0-logloss:0.68921
[1]	validation_0-logloss:0.68507
[2]	validation_0-logloss:0.68126
[3]	validation_0-logloss:0.67732
[4]	validation_0-logloss:0.67260
[5]	validation_0-logloss:0.66873
[6]	validation_0-logloss:0.66405
[7]	validation_0-logloss:0.65966
[8]	validation_0-logloss:0.65562
[9]	validation_0-logloss:0.65204
[10]	validation_0-logloss:0.64815
[11]	validation_0-logloss:0.64398
[12]	validation_0-logloss:0.64027
[13]	validation_0-logloss:0.63645
[14]	validation_0-logloss:0.63237
[15]	validation_0-logloss:0.62943
[16]	validation_0-logloss:0.62584
[17]	validation_0-logloss:0.62310
[18]	validation_0-logloss:0.61913
[19]	validation_0-logloss:0.61560
[20]	validation_0-logloss:0.61218
[21]	validation_0-logloss:0.60934
[22]	validation_0-logloss:0.60605
[23]	validation_0-logloss:0.60343
[24]	validation_0-logloss:0.60043
[25]	validation_0-logloss:0.59687
[26]	validation_0-logloss:0.59372
[27]	validation_0-logloss:0.59036
[28]	validation_0-logloss:0.58756
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.009, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=1,
              num_parallel_tree=None, ...)

In [35]:
# Evaluate on Dataset B test set
y_pred_baseline = baseline_model.predict(X_finetune_test)

baseline_acc = accuracy_score(y_finetune_test, y_pred_baseline) * 100
baseline_f1  = f1_score(y_finetune_test, y_pred_baseline) * 100

cm = confusion_matrix(y_finetune_test, y_pred_baseline)
TN, FP, FN, TP = cm.ravel()

recall_score_calc = TP / (TP + FN) * 100
specificity_score_calc = TN / (TN + FP) * 100


print(f"Baseline Accuracy (Dataset B): {baseline_acc:.2f}%")
print(f"Baseline F1-score (Dataset B): {baseline_f1:.2f}%")
print(f"Baseline Recall (Sensitivity): {recall_score_calc:.2f}%")
print(f"Baseline Specificity: {specificity_score_calc:.2f}%")

Baseline Accuracy (Dataset B): 83.87%
Baseline F1-score (Dataset B): 83.87%
Baseline Recall (Sensitivity): 92.86%
Baseline Specificity: 76.47%


## 6. Tabnet

In [36]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [37]:
# Defining the training, validation, and test datasets for the source domain
X_A_train = X_pretrain_train_res.values  # Features for training on the source domain (pre-training phase)
y_A_train = y_pretrain_train_res.values  # Target labels for training on the source domain

X_A_val   = X_pretrain_val.values  # Features for validation on the source domain
y_A_val   = y_pretrain_val.values  # Target labels for validation on the source domain

X_A_test  = X_pretrain_test.values  # Features for testing on the source domain
y_A_test  = y_pretrain_test.values  # Target labels for testing on the source domain

# Defining the training, validation, and test datasets for the target domain
X_B_train = X_finetune_train_res.values  # Features for training on the target domain (fine-tuning phase)
y_B_train = y_finetune_train_res.values  # Target labels for training on the target domain

X_B_val   = X_finetune_val.values  # Features for validation on the target domain
y_B_val   = y_finetune_val.values  # Target labels for validation on the target domain

X_B_test  = X_finetune_test.values  # Features for testing on the target domain
y_B_test  = y_finetune_test.values  # Target labels for testing on the target domain

In [38]:
scaler = StandardScaler()

X_A_train = scaler.fit_transform(X_A_train).astype(np.float32)  # Fit scaler on source train set, transform it
X_A_val = scaler.transform(X_A_val).astype(np.float32)          # Transform source validation set
X_A_test = scaler.transform(X_A_test).astype(np.float32)        # Transform source test set

y_A_train = y_A_train.astype(np.int64)  # Convert source train labels to int64
y_A_val = y_A_val.astype(np.int64)      # Convert source validation labels
y_A_test = y_A_test.astype(np.int64)    # Convert source test labels

X_B_train = scaler.transform(X_B_train).astype(np.float32)      # Apply source-domain scaling to target train set
X_B_val = scaler.transform(X_B_val).astype(np.float32)          # Apply scaling to target validation set
X_B_test = scaler.transform(X_B_test).astype(np.float32)        # Apply scaling to target test set

y_B_train = y_B_train.astype(np.int64)  # Convert target train labels to int64
y_B_val = y_B_val.astype(np.int64)      # Convert target validation labels
y_B_test = y_B_test.astype(np.int64)    # Convert target test labels

### Pretraining on the source domain

#### Hyperparameter tuning for Pre-training

In [94]:
# Objective function to optimize
def objective(trial):
# n_d and n_a (often kept equal for simplicity)
    n_d_a = trial.suggest_int('n_d_a', 8, 32, step=8) 
    n_steps = trial.suggest_int('n_steps', 4, 10) 
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-6, 1e-2, log=True)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float('momentum', 0.01, 0.4, step=0.01)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [16, 32])
    step_size = trial.suggest_int('step_size', 20, 80, step=10)
    scheduler_gamma = trial.suggest_float('scheduler_gamma', 0.8, 0.98)

    tabnet_model = TabNetClassifier(
        n_d=n_d_a,
        n_a=n_d_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        momentum=momentum,
        optimizer_fn=torch.optim.Adam,
        optimizer_params={'lr': lr},
        mask_type='sparsemax',
        scheduler_params={'step_size': step_size, 'gamma': scheduler_gamma},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=0, # Crucial to silence output during 50+ trials
        seed=42
    )

    tabnet_model.fit(
        X_train=X_A_train, 
        y_train=y_A_train, 
        eval_set=[(X_A_val, y_A_val)], 
        eval_name=["val"], 
        # Use a supported metric for early stopping
        eval_metric=["balanced_accuracy"], 
        max_epochs=100, 
        patience=20, # Reduced patience for faster tuning
        batch_size=batch_size, 
        virtual_batch_size=virtual_batch_size
    )
    
    # Predict and calculate the final metric to maximize
    preds = tabnet_model.predict(X_A_val)
    f1 = f1_score(y_A_val, preds)

    accuracy = accuracy_score(y_A_val, preds)
    
    # Print the results of the current trial for monitoring
    print(f"Trial {trial.number} results: Accuracy = {accuracy:.4f}, F1-Score = {f1:.4f}")
        
    return f1 

# --- Run the Optimization ---
study_scratch = optuna.create_study(direction='maximize')
study_scratch.optimize(objective, n_trials=30) # Recommend a higher trial count for full tuning

[I 2025-12-29 22:50:19,621] A new study created in memory with name: no-name-bbda8a31-e4aa-4bbd-8bbc-d30a736d7e4c



Early stopping occurred at epoch 32 with best_epoch = 12 and best_val_balanced_accuracy = 0.81588


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-29 22:50:45,741] Trial 0 finished with value: 0.8235294117647058 and parameters: {'n_d_a': 8, 'n_steps': 10, 'gamma': 1.5213303366613837, 'lambda_sparse': 5.410389733001134e-06, 'lr': 0.01342596619210637, 'momentum': 0.24000000000000002, 'batch_size': 64, 'virtual_batch_size': 32, 'step_size': 80, 'scheduler_gamma': 0.818499861081206}. Best is trial 0 with value: 0.8235294117647058.


Trial 0 results: Accuracy = 0.8033, F1-Score = 0.8235

Early stopping occurred at epoch 59 with best_epoch = 39 and best_val_balanced_accuracy = 0.52083


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-29 22:51:06,406] Trial 1 finished with value: 0.7628865979381443 and parameters: {'n_d_a': 16, 'n_steps': 6, 'gamma': 1.2169440085479426, 'lambda_sparse': 0.00010244960546271797, 'lr': 1.7637096255912015e-05, 'momentum': 0.17, 'batch_size': 128, 'virtual_batch_size': 16, 'step_size': 60, 'scheduler_gamma': 0.8143267286746101}. Best is trial 0 with value: 0.8235294117647058.


...

Trial 27 results: Accuracy = 0.8361, F1-Score = 0.8611

Early stopping occurred at epoch 41 with best_epoch = 21 and best_val_balanced_accuracy = 0.87613


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-29 23:59:20,799] Trial 28 finished with value: 0.9066666666666666 and parameters: {'n_d_a': 32, 'n_steps': 7, 'gamma': 1.8698558098046372, 'lambda_sparse': 0.005393173126575441, 'lr': 0.04179737727422789, 'momentum': 0.27, 'batch_size': 64, 'virtual_batch_size': 16, 'step_size': 30, 'scheduler_gamma': 0.9421521438008308}. Best is trial 28 with value: 0.9066666666666666.


Trial 28 results: Accuracy = 0.8852, F1-Score = 0.9067

Early stopping occurred at epoch 31 with best_epoch = 11 and best_val_balanced_accuracy = 0.82207


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-29 23:59:43,820] Trial 29 finished with value: 0.8450704225352113 and parameters: {'n_d_a': 32, 'n_steps': 10, 'gamma': 1.4776803433276946, 'lambda_sparse': 7.632793437647826e-06, 'lr': 0.05179155526814573, 'momentum': 0.26, 'batch_size': 64, 'virtual_batch_size': 32, 'step_size': 80, 'scheduler_gamma': 0.9052827819153031}. Best is trial 28 with value: 0.9066666666666666.


Trial 29 results: Accuracy = 0.8197, F1-Score = 0.8451


In [95]:
# Get best trial and parameters
best_trial = study_scratch.best_trial
best_pretrain_params = best_trial.params

print("Best pretrain params:")
print(f"  Value (F1-score): {best_trial.value:.4f}")
print("  Params:")

for key, value in best_pretrain_params.items():
    print(f"    {key}: {value}")

Best pretrain params:
  Value (F1-score): 0.9067
  Params:
    n_d_a: 32
    n_steps: 7
    gamma: 1.8698558098046372
    lambda_sparse: 0.005393173126575441
    lr: 0.04179737727422789
    momentum: 0.27
    batch_size: 64
    virtual_batch_size: 16
    step_size: 30
    scheduler_gamma: 0.9421521438008308


#### Pre-training on the source domain

In [76]:
# Define the best parameters found during the Grid Search for pretraining on Dataset A......DO NOT RUN THIS CODE IF YOU DO HYPERPARAMETER TUNING
best_pretrain_params = {
    "n_d_a": 16,
    "n_steps": 9,
    "gamma": 1.7899467948557366,
    "lambda_sparse": 4.387608964907318e-05,
    "lr": 0.003074702431245894,
    "momentum": 0.22,
    "batch_size": 32,
    "virtual_batch_size": 32,
    "step_size": 40,
    "scheduler_gamma": 0.9464940708357479,
}
for key, value in best_pretrain_params.items():
    print(f"{key}: {value}")

n_d_a: 16
n_steps: 9
gamma: 1.7899467948557366
lambda_sparse: 4.387608964907318e-05
lr: 0.003074702431245894
momentum: 0.22
batch_size: 32
virtual_batch_size: 32
step_size: 40
scheduler_gamma: 0.9464940708357479


In [77]:
# .....UNCOMMENT THIS LINE IF YOU DO HYPERPARAMETER TUNING
# best_pretrain_params = study_scratch.best_params

tabnet_pretrain = TabNetClassifier(
    n_d=best_pretrain_params['n_d_a'],           # split n_d_a into n_d
    n_a=best_pretrain_params['n_d_a'],           # and n_a
    n_steps=best_pretrain_params['n_steps'],
    gamma=best_pretrain_params['gamma'],
    lambda_sparse=best_pretrain_params['lambda_sparse'],
    momentum=best_pretrain_params['momentum'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params={'lr': best_pretrain_params['lr']},
    mask_type='sparsemax',
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={'step_size': best_pretrain_params['step_size'], 'gamma': best_pretrain_params['scheduler_gamma']},
    verbose=1,
    seed=42
)


# Train the final model
tabnet_pretrain.fit(
    X_train=X_A_train,
    y_train=y_A_train,
    eval_set=[(X_A_val, y_A_val)],
    eval_name=["val"],
    eval_metric=["logloss"],
    max_epochs=150,
    patience=30,
    batch_size=best_pretrain_params['batch_size'],
    virtual_batch_size=best_pretrain_params['virtual_batch_size'],
    num_workers=0,
    drop_last=False
)

# Save the trained model
tabnet_pretrain.save_model("tabnet_pretrain_model")

C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.21364 | val_logloss: 1.03659 |  0:00:01s
epoch 1  | loss: 0.79312 | val_logloss: 0.87236 |  0:00:03s
epoch 2  | loss: 0.65424 | val_logloss: 0.55415 |  0:00:04s
epoch 3  | loss: 0.6064  | val_logloss: 0.69821 |  0:00:06s
epoch 4  | loss: 0.53383 | val_logloss: 0.55025 |  0:00:08s
epoch 5  | loss: 0.4903  | val_logloss: 0.49206 |  0:00:09s
epoch 6  | loss: 0.50623 | val_logloss: 0.46262 |  0:00:11s
epoch 7  | loss: 0.4663  | val_logloss: 0.53205 |  0:00:13s
epoch 8  | loss: 0.46921 | val_logloss: 0.58841 |  0:00:15s
epoch 9  | loss: 0.47955 | val_logloss: 0.59093 |  0:00:16s
epoch 10 | loss: 0.46567 | val_logloss: 0.53411 |  0:00:18s
epoch 11 | loss: 0.42929 | val_logloss: 0.44199 |  0:00:19s
epoch 12 | loss: 0.43843 | val_logloss: 0.53864 |  0:00:21s
epoch 13 | loss: 0.43684 | val_logloss: 0.48473 |  0:00:22s
epoch 14 | loss: 0.40049 | val_logloss: 0.46563 |  0:00:24s
epoch 15 | loss: 0.41974 | val_logloss: 0.55322 |  0:00:25s
epoch 16 | loss: 0.40758 | val_logloss: 

C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_pretrain_model.zip


'tabnet_pretrain_model.zip'

In [78]:
Dataset_A_Pretrain_TNet = tabnet_pretrain.predict(X_A_test)

cm_A_TNet = confusion_matrix(y_A_test, Dataset_A_Pretrain_TNet)
TN_A, FP_A, FN_A, TP_A = cm_A_TNet.ravel()

# Calculate Metrics for TabNet on Dataset A
acc_A_TNet = accuracy_score(y_A_test, Dataset_A_Pretrain_TNet) * 100
f1_A_TNet = f1_score(y_A_test, Dataset_A_Pretrain_TNet) * 100
recall_A_TNet = TP_A / (TP_A + FN_A) * 100
specificity_A_TNet = TN_A / (TN_A + FP_A) * 100


print(f"Pre-training, test on Dataset A, Test Accuracy: {acc_A_TNet:.2f}%")
print(f"Pre-training, test on Dataset A, Test F1-score: {f1_A_TNet:.2f}%")
print(f"Pre-training, test on Dataset A, Test Recall (Sensitivity): {recall_A_TNet:.2f}%")
print(f"Pre-training, test on Dataset A, Test Specificity: {specificity_A_TNet:.2f}%")

Pre-training, test on Dataset A, Test Accuracy: 83.87%
Pre-training, test on Dataset A, Test F1-score: 86.49%
Pre-training, test on Dataset A, Test Recall (Sensitivity): 86.49%
Pre-training, test on Dataset A, Test Specificity: 80.00%


In [79]:
Dataset_B_Pretrain_TNet = tabnet_pretrain.predict(X_B_test)

# Calculate Confusion Matrix for TabNet on Dataset B
# Structure: [[TN, FP], [FN, TP]]
cm_B_TNet = confusion_matrix(y_B_test, Dataset_B_Pretrain_TNet)
TN_B, FP_B, FN_B, TP_B = cm_B_TNet.ravel()

# Calculate Core Metrics
acc_B_TNet = accuracy_score(y_B_test, Dataset_B_Pretrain_TNet) * 100
f1_B_TNet = f1_score(y_B_test, Dataset_B_Pretrain_TNet) * 100

# Calculate Recall (Sensitivity) and Specificity
recall_B_TNet = TP_B / (TP_B + FN_B) * 100
specificity_B_TNet = TN_B / (TN_B + FP_B) * 100

print(f"Pre-training, test on Dataset B, Accuracy: {acc_B_TNet:.2f}%")
print(f"Pre-training, test on Dataset B, F1-score: {f1_B_TNet:.2f}%")
print(f"Pre-training, test on Dataset B, Recall (Sensitivity): {recall_B_TNet:.2f}%")
print(f"Pre-training, test on Dataset B, Specificity: {specificity_B_TNet:.2f}%")


Pre-training, test on Dataset B, Accuracy: 74.19%
Pre-training, test on Dataset B, F1-score: 73.33%
Pre-training, test on Dataset B, Recall (Sensitivity): 78.57%
Pre-training, test on Dataset B, Specificity: 70.59%


#### Hyperparameter tuning for fine-tuning

In [106]:
def objective_finetune(trial):

    # --- Hyperparameters to tune ---
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-6, 1e-2, log=True)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    momentum = trial.suggest_float('momentum', 0.01, 0.4, step=0.01)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)

    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [16, 32])

    step_size = trial.suggest_int('step_size', 20, 80, step=10)
    scheduler_gamma = trial.suggest_float('scheduler_gamma', 0.8, 0.98)

    # --- Model definition ---
    tabnet_model_finetune = TabNetClassifier(
        n_d=best_pretrain_params['n_d_a'],
        n_a=best_pretrain_params['n_d_a'],
        n_steps=best_pretrain_params['n_steps'],
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        momentum=momentum,
        optimizer_fn=torch.optim.Adam,
        optimizer_params={
            'lr': lr,
        },
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        scheduler_params={
            'step_size': step_size,
            'gamma': scheduler_gamma
        },
        mask_type='entmax',
        verbose=0,
        seed=42,
        device_name='cuda' if torch.cuda.is_available() else 'cpu'
    )

    # --- Fine-tuning ---
    tabnet_model_finetune.fit(
        X_train=X_B_train,
        y_train=y_B_train,
        eval_set=[(X_B_val, y_B_val)],
        eval_name=["val"],
        eval_metric=["logloss"],
        max_epochs=100,
        patience=40,  # FIXED (not optimized)
        batch_size=batch_size,
        virtual_batch_size=virtual_batch_size,
        num_workers=0,
        drop_last=False,
        from_unsupervised=tabnet_pretrain
    )

    # --- Evaluation ---
    preds = tabnet_model_finetune.predict(X_B_val)
    f1 = f1_score(y_B_val, preds)
    acc = accuracy_score(y_B_val, preds)

    print(
        f"Trial {trial.number} | "
        f"F1: {f1:.4f} | Acc: {acc:.4f} | "
        f"LR: {lr:.1e} | Mom: {momentum:.2f}"
    )

    return f1


# --- Run the Optimization ---
study_finetune = optuna.create_study(direction='maximize')  # We want to maximize F1-Score
study_finetune.optimize(objective_finetune, n_trials=30)  # Number of trials to run for optimization


[I 2025-12-30 00:08:46,950] A new study created in memory with name: no-name-bab354e7-25a7-47e6-b959-3fb7edff0247
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from entmax to sparsemax
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-30 00:09:05,180] Trial 0 finished with value: 0.9333333333333333 and parameters: {'gamma': 1.2422921413473458, 'lambda_sparse': 4.4674843238322395e-05, 'lr': 0.0005454191359802393, 'momentum': 0.06999999999999999, 'weight_decay': 0.00021888834807352566, 'batch_size': 128, 'virtual_batch_size': 16, 'step_size': 30, 'scheduler_g


Early stopping occurred at epoch 76 with best_epoch = 36 and best_val_logloss = 0.17214


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from entmax to sparsemax
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-30 00:09:18,424] Trial 1 finished with value: 0.9032258064516129 and parameters: {'gamma': 1.1143049288356874, 'lambda_sparse': 2.4633424661838425e-05, 'lr': 1.4170275794756587e-05, 'momentum': 0.24000000000000002, 'weight_decay': 0.0008719987291601801, 'batch_size': 128, 'virtual_batch_size': 32, 'step_size': 50, 'scheduler_gamma': 0.9445796584268462}. Best is trial 0 with value: 0.9333333333333333.



Early stopping occurred at epoch 71 with best_epoch = 31 and best_val_logloss = 0.26891


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from entmax to sparsemax
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")


...

C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from entmax to sparsemax
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-30 00:20:33,298] Trial 28 finished with value: 0.9333333333333333 and parameters: {'gamma': 1.4529768377094345, 'lambda_sparse': 5.64854915071735e-05, 'lr': 0.004418840121748493, 'momentum': 0.27, 'weight_decay': 0.0004359578007004053, 'batch_size': 64, 'virtual_batch_size': 16, 'step_size': 50, 'scheduler_gamma': 0.9773694610251796}. Best is trial 8 with value: 0.9655172413793104.



Early stopping occurred at epoch 42 with best_epoch = 2 and best_val_logloss = 0.16867


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from entmax to sparsemax
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 55 with best_epoch = 15 and best_val_logloss = 0.16513


C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-30 00:21:01,868] Trial 29 finished with value: 0.9655172413793104 and parameters: {'gamma': 1.1505639134565493, 'lambda_sparse': 2.424671090397197e-05, 'lr': 0.0007412784697649567, 'momentum': 0.36000000000000004, 'weight_decay': 0.00014598653267065606, 'batch_size': 32, 'virtual_batch_size': 16, 'step_size': 60, 'scheduler_gamma': 0.8363723847926725}. Best is trial 8 with value: 0.9655172413793104.


In [107]:
# Get best trial and parameters
best_trial_finetune = study_finetune.best_trial
best_finetune_params = best_trial_finetune.params

print("Best finetune params:")
print(f"  Value (F1-score): {best_trial_finetune.value:.4f}")
print("  Params:")

for key, value in best_finetune_params.items():
    print(f"    {key}: {value}")

Best finetune params:
  Value (F1-score): 0.9655
  Params:
    gamma: 1.0093336159088
    lambda_sparse: 0.000287623142768745
    lr: 0.0005151997387371908
    momentum: 0.29000000000000004
    weight_decay: 0.00035940837944500203
    batch_size: 64
    virtual_batch_size: 16
    step_size: 70
    scheduler_gamma: 0.8291178754457263


#### Finetunine on the target domain

In [80]:
best_finetune_params = {
    "gamma": 1.87,  
    "lambda_sparse": 0.00176,  
    "lr": 0.00236,  
    "momentum": 0.92, 
    "batch_size": 32,  
    "virtual_batch_size": 32,  
    "step_size": 60,  
    "scheduler_gamma": 0.946, 
}

# Print the updated values
for key, value in best_finetune_params.items():
    print(f"{key}: {value}")


gamma: 1.87
lambda_sparse: 0.00176
lr: 0.00236
momentum: 0.92
batch_size: 32
virtual_batch_size: 32
step_size: 60
scheduler_gamma: 0.946


In [87]:
# Uncomment the line below if you do hyperparameter tuning
# best_finetune_params = study_finetune.best_params

# Create a new model for fine-tuning
tabnet_finetuned = TabNetClassifier(
    n_d=8,
    n_a=8,
    n_steps=3,
    gamma=best_finetune_params['gamma'],
    lambda_sparse=best_finetune_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params={
        'lr': best_finetune_params['lr']
    },
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={'step_size': best_finetune_params['step_size'], 'gamma': best_finetune_params['scheduler_gamma']},
    seed=42,
    verbose=1,
    device_name='cuda' if torch.cuda.is_available() else 'cpu'
)
tabnet_finetuned.fit(
    X_train=X_B_train,
    y_train=y_B_train,           
    eval_set=[(X_B_val, y_B_val)],
    eval_name=["val"],
    eval_metric=['logloss'],
    max_epochs=100,
    patience=40,
    batch_size=best_finetune_params['batch_size'],
    virtual_batch_size=best_finetune_params['virtual_batch_size'],
    num_workers=0,
    drop_last=False,
    # Initialize with pre-trained weights
    from_unsupervised=tabnet_pretrain  # Transfer learning!
)

C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_a changed from 8 to 16
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_d changed from 8 to 16
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_steps changed from 3 to 9
  warnings.warn(wrn_msg)
C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")


epoch 0  | loss: 0.65235 | val_logloss: 0.24879 |  0:00:00s
epoch 1  | loss: 0.59593 | val_logloss: 0.2366  |  0:00:01s
epoch 2  | loss: 0.58144 | val_logloss: 0.22615 |  0:00:02s
epoch 3  | loss: 0.485   | val_logloss: 0.30044 |  0:00:03s
epoch 4  | loss: 0.60511 | val_logloss: 0.3088  |  0:00:04s
epoch 5  | loss: 0.48286 | val_logloss: 0.23589 |  0:00:04s
epoch 6  | loss: 0.45735 | val_logloss: 0.16189 |  0:00:05s
epoch 7  | loss: 0.49516 | val_logloss: 0.16318 |  0:00:06s
epoch 8  | loss: 0.48312 | val_logloss: 0.1911  |  0:00:07s
epoch 9  | loss: 0.51852 | val_logloss: 0.19563 |  0:00:08s
epoch 10 | loss: 0.41846 | val_logloss: 0.20051 |  0:00:08s
epoch 11 | loss: 0.45739 | val_logloss: 0.18843 |  0:00:09s
epoch 12 | loss: 0.45904 | val_logloss: 0.25945 |  0:00:10s
epoch 13 | loss: 0.43297 | val_logloss: 0.17831 |  0:00:11s
epoch 14 | loss: 0.43837 | val_logloss: 0.28718 |  0:00:11s
epoch 15 | loss: 0.44192 | val_logloss: 0.29492 |  0:00:12s
epoch 16 | loss: 0.46761 | val_logloss: 

C:\Users\prosp\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [88]:
Dataset_A_Finetune_TNet = tabnet_finetuned.predict(X_A_test)

# Calculate Confusion Matrix for Fine-Tuned TabNet on Dataset A
# Structure: [[TN, FP], [FN, TP]]
cm_A_Finetune_TNet = confusion_matrix(y_A_test, Dataset_A_Finetune_TNet)
TN_A_F, FP_A_F, FN_A_F, TP_A_F = cm_A_Finetune_TNet.ravel()

# Calculate Core Metrics
acc_A_Finetune_TNet = accuracy_score(y_A_test, Dataset_A_Finetune_TNet) * 100
f1_A_Finetune_TNet = f1_score(y_A_test, Dataset_A_Finetune_TNet) * 100

# Calculate Recall (Sensitivity) and Specificity
recall_A_Finetune_TNet = TP_A_F / (TP_A_F + FN_A_F) * 100 if (TP_A_F + FN_A_F) > 0 else 0
specificity_A_Finetune_TNet = TN_A_F / (TN_A_F + FP_A_F) * 100 if (TN_A_F + FP_A_F) > 0 else 0

print(f"Fine-tuned, test on Dataset A, Test Accuracy: {acc_A_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset A, Test F1-score: {f1_A_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset A, Test Recall (Sensitivity): {recall_A_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset A, Test Specificity: {specificity_A_Finetune_TNet:.2f}%")

Fine-tuned, test on Dataset A, Test Accuracy: 82.26%
Fine-tuned, test on Dataset A, Test F1-score: 84.51%
Fine-tuned, test on Dataset A, Test Recall (Sensitivity): 81.08%
Fine-tuned, test on Dataset A, Test Specificity: 84.00%


In [89]:
Dataset_B_Finetune_TNet = tabnet_finetuned.predict(X_B_test)

# Calculate Confusion Matrix for Fine-Tuned TabNet on Dataset B
# Structure: [[TN, FP], [FN, TP]]
cm_B_Finetune_TNet = confusion_matrix(y_B_test, Dataset_B_Finetune_TNet)
TN_F, FP_F, FN_F, TP_F = cm_B_Finetune_TNet.ravel()

# Calculate Core Metrics
acc_B_Finetune_TNet = accuracy_score(y_B_test, Dataset_B_Finetune_TNet) * 100
f1_B_Finetune_TNet = f1_score(y_B_test, Dataset_B_Finetune_TNet) * 100

# Calculate Recall (Sensitivity) and Specificity
recall_B_Finetune_TNet = TP_F / (TP_F + FN_F) * 100
specificity_B_Finetune_TNet = TN_F / (TN_F + FP_F) * 100

print(f"Fine-tuned, test on Dataset B, Accuracy: {acc_B_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset B, F1-score: {f1_B_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset B, Recall (Sensitivity): {recall_B_Finetune_TNet:.2f}%")
print(f"Fine-tuned, test on Dataset B, Specificity: {specificity_B_Finetune_TNet:.2f}%")

Fine-tuned, test on Dataset B, Accuracy: 70.97%
Fine-tuned, test on Dataset B, F1-score: 70.97%
Fine-tuned, test on Dataset B, Recall (Sensitivity): 78.57%
Fine-tuned, test on Dataset B, Specificity: 64.71%


In [ ]:
# Set global font to Times New Roman
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size' : '16'})

cm = confusion_matrix(y_finetune_test, Dataset_B_Finetune)

# Unravel the confusion matrix to get TN, FP, FN, TP
tn, fp, fn, tp = cm.ravel()

# Create the plot with a specific size
plt.figure(figsize=(5, 4))

# Display confusion matrix as an image with the 'Blues' color map
im = plt.imshow(cm, cmap='Blues', interpolation='nearest')

# Set color bar ticks based on the max value of the matrix
vmax = cm.max()
colorbar_ticks = np.arange(0, vmax + 10, 10)
plt.colorbar(im, ticks=colorbar_ticks)

# Remove title as per request (You can uncomment the next line if you want to add a title)
# plt.title('Confusion Matrix of the fine-tuned model')

# Set X-axis labels (Predicted: Disease then No Disease)
plt.xticks([0, 1], ['No Disease', 'Disease']) 
plt.yticks([0, 1], ['No Disease', 'Disease']) 

# Label the axes
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Normalize the color range based on the confusion matrix values
norm = mcolors.Normalize(vmin=cm.min(), vmax=cm.max())
cmap = plt.colormaps['Blues']

# Iterate through the matrix to add text inside each cell
for i in range(2):
    for j in range(2):
        cell_value = cm[i, j]
        cell_color = cmap(norm(cell_value))  # Get the color of the cell based on the value
        luminance = 0.299 * cell_color[0] + 0.587 * cell_color[1] + 0.114 * cell_color[2]  # Calculate luminance
        text_color = 'white' if luminance < 0.5 else 'black'  # Choose white or black text based on luminance
        
        # Add the text inside the cell with bold and Times New Roman font
        plt.text(j, i, f'{cell_value}', ha='center', va='center', color=text_color, fontsize=24, fontweight='bold')

# Save the confusion matrix image with high resolution
plt.savefig('confusion_matrix_tabnet.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
# Ensure Times New Roman font is used in all plots
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size' : '20'})

# Get predictions for TabNet model
y_pred = tabnet_finetuned.predict(X_B_test)  # Get the class predictions for target domain
y_pred_prob = tabnet_finetuned.predict_proba(X_B_test)[:, 1]  # Probabilities for the positive class (ROC)

# Calculate Precision, Recall, F1-Score
precision = precision_score(y_B_test, y_pred) * 100
recall = recall_score(y_B_test, y_pred) * 100
f1 = f1_score(y_B_test, y_pred) * 100

# Display the metrics
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1-Score: {f1:.2f}%")

# Calculate Specificity: TN / (TN + FP)
tn, fp, fn, tp = confusion_matrix(y_B_test, y_pred).ravel()
specificity = (tn / (tn + fp)) * 100
print(f"Specificity: {specificity:.2f}%")

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_B_test, y_pred_prob) * 100
print(f"ROC-AUC: {roc_auc:.2f}%")

# Plot ROC curve with blue tones (IEEE standard)
fpr, tpr, _ = roc_curve(y_B_test, y_pred_prob)
roc_auc_value = auc(fpr, tpr)

plt.figure(figsize=(7, 6))

# Plot the ROC curve: Use 'orange' color and update the label
plt.plot(fpr, tpr, color='orange', lw=2, label=f'ROC curve (AUC = {roc_auc_value:.2f})')
plt.plot([0, 1], [0, 1], color='darkblue', lw=2, linestyle='--')

# Set limits and labels as in the original image
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)

# Adjust legend location to match the original image
plt.legend(loc='lower right', fontsize=18)

# Save the ROC curve image (you can keep your original filename or update it)
plt.savefig('roc_curve_tabnet.png', dpi=300, bbox_inches='tight')
plt.show()  # Display the plot



## 7. MLP

### Pretraining on the source domain

#### Hyperparameter tuning for pretraining stage

In [93]:
def build_model_for_tuning(hp):
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_l2_reg = hp.Choice('l2_reg', values=[0.01, 0.001, 0.0001])
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    hp_neurons_l1 = hp.Int('neurons_l1', min_value=32, max_value=128, step=32)

    # input_dim = X_pretrain_train_res.shape[1]
    input_dim = X_A_train.shape[1]
    inputs = Input(shape=(input_dim,))

    # Layer 1 (Tuned Neurons, Tuned Dropout, Tuned L2)
    x = Dense(
        hp_neurons_l1, 
        activation='relu', 
        name='feature_layer_1',
        kernel_regularizer=keras.regularizers.l2(hp_l2_reg)
    )(inputs)
    x = Dropout(hp_dropout_rate)(x)

    # Layer 2 (Fixed at half of L1, or can be tuned as well)
    x = Dense(
        int(hp_neurons_l1 / 2), # e.g., 64 -> 32
        activation='relu', 
        name='feature_layer_2',
        kernel_regularizer=keras.regularizers.l2(hp_l2_reg)
    )(x)
    x = Dropout(hp_dropout_rate)(x)
    
    # Layer 3 (Fixed at quarter of L1, or can be tuned)
    x = Dense(
        int(hp_neurons_l1 / 4), # e.g., 64 -> 16
        activation='relu', 
        name='feature_layer_3',
        kernel_regularizer=keras.regularizers.l2(hp_l2_reg)
    )(x)
    x = Dropout(hp_dropout_rate)(x)
    
    # Output Layer
    outputs = Dense(1, activation='sigmoid', name='output_layer')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc')] 
    )
    return model

In [94]:
print(X_pretrain_val.shape)
print(X_pretrain_train_res.shape)  # should match input layer


(61, 10)
(590, 10)


In [95]:
tuner = kt.RandomSearch(
    build_model_for_tuning,
    objective='auc',  # Metric to MAXIMIZE during tuning
    max_trials=30,        # Number of different models/hyperparameter combos to test
    executions_per_trial=3, # Number of times to train each model to reduce randomness
    directory='mlp_tuning', 
    project_name='heart_disease_pretrain'
)

# Optional: Print the search space
tuner.search_space_summary()

# Run the search
tuner.search(
    # X_pretrain_train_res, 
    # y_pretrain_train_res,
    X_A_train, 
    y_A_train,
    epochs=80, 
    validation_data=(X_A_val, y_A_val),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ],
    verbose=1
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\n--- Optimal Hyperparameters ---")
print(f"Learning Rate: {best_hps.get('learning_rate')}")
print(f"L2 Reg: {best_hps.get('l2_reg')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Neurons L1: {best_hps.get('neurons_l1')}")

# Retrieve the best model found
best_model = tuner.get_best_models(num_models=1)[0]

Search space summary
Default search space size: 4
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
l2_reg (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
neurons_l1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |learning_rate
0.001             |0.001             |l2_reg
0.1               |0.1               |dropout_rate
96                |96                |neurons_l1

Epoch 1/80
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.7508 - auc: 0.8465 - loss: 0.6890 - val_accuracy: 0.7541 - val_auc: 0.8198 - val_loss: 0.6453
Epoch 2/80
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - 

KeyboardInterrupt: 

In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

best_params = best_hp.get_config()['values']

print("--- Best Hyperparameters ---")
for param, value in best_params.items():
    print(f"{param}: {value}")

#### Pretraining on the source domain

In [96]:
#DO NOT RUN THIS LINE IF YOU DO THE HYPERPARAMETER TUNING
best_params = {
    'learning_rate' : 0.01,
    'l2_reg' : 0.0001,
    'neurons_l1' : 96,
    'dropout_rate' : 0.1
}

In [97]:
def create_mlp_model(input_shape, best_params):
    neurons_l1 = best_params['neurons_l1']
    dropout_rate = best_params['dropout_rate']
    l2_reg = best_params['l2_reg']
    learning_rate = best_params['learning_rate']

    inputs = Input(shape=(input_shape,))
    x = Dense(neurons_l1, activation='relu', name='feature_layer_1', kernel_regularizer=keras.regularizers.l2(l2_reg))(inputs)
    x = Dropout(dropout_rate)(x)
    
    x = Dense(int(neurons_l1/2), activation='relu', name='feature_layer_2', kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = Dropout(dropout_rate)(x)
    
    x = Dense(int(neurons_l1/4), activation='relu', name='feature_layer_3', kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = Dropout(dropout_rate)(x)
    
    outputs = Dense(1, activation='sigmoid', name='output_layer')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc')]
    )
    return model

# input_dim = X_pretrain_train_res.shape[1]
input_dim = X_A_train.shape[1]
mlp_pretrain = create_mlp_model(input_dim, best_params)
mlp_pretrain.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_1 (Dense)              │ (None, 96)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_2 (Dense)              │ (None, 48)                  │           4,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_3 (Dense)              │ (None, 24)                  │           1,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,913 (27.00 KB)

 Trainable params: 6,913 (27.00 KB)

 Non-trainable params: 0 (0.00 B)

In [98]:
pretrain_history = mlp_pretrain.fit(
    # X_pretrain_train_res, 
    # y_pretrain_train_res,
    X_A_train, 
    y_A_train,
    epochs=150, # Start with a reasonable number of epochs
    batch_size=32,
    # validation_data=(X_pretrain_val, y_pretrain_val),
    validation_data=(X_A_val, y_A_val),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=52, restore_best_weights=True)
    ],
    shuffle=False,
    verbose=1
)

# Save the pre-trained weights using the required extension
mlp_pretrain.save_weights('mlp_pretrain.weights.h5')

Epoch 1/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.8390 - auc: 0.9066 - loss: 0.4111 - val_accuracy: 0.7869 - val_auc: 0.9048 - val_loss: 0.4519
Epoch 2/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8610 - auc: 0.9274 - loss: 0.3553 - val_accuracy: 0.8361 - val_auc: 0.9026 - val_loss: 0.3843
Epoch 3/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8847 - auc: 0.9483 - loss: 0.3060 - val_accuracy: 0.8525 - val_auc: 0.9178 - val_loss: 0.3508
Epoch 4/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9000 - auc: 0.9607 - loss: 0.2710 - val_accuracy: 0.8197 - val_auc: 0.9093 - val_loss: 0.3510
Epoch 5/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8949 - auc: 0.9660 - loss: 0.2541 - val_accuracy: 0.8361 - val_auc: 0.9105 - val_loss: 0.3537
Epoch 6/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9169 - auc: 0.9719 - loss: 0.2257 - val_accuracy: 0.8525 - val_auc: 0.9178 - val_loss: 0.3558
Epoch 7/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms

In [99]:
# Use the fine-tune validation set (X_finetune_val) to find the threshold
y_val_proba = mlp_pretrain.predict(X_B_val).ravel()
precision, recall, thresholds = precision_recall_curve(y_B_val, y_val_proba)

# Calculate F1 score for all thresholds
fscore = (2 * precision * recall) / (precision + recall + 1e-6) # Added 1e-6 to prevent division by zero
# Find the threshold that yields the maximum F1 score
ix = np.argmax(fscore)
best_threshold = thresholds[ix]

print(f"Optimal Threshold (Max F1 on Validation): {best_threshold:.4f}")
print(f"Max F1 Score at this threshold: {fscore[ix]:.4f}")

# Now, use best_threshold instead of 0.5 for final testing (e.g., y_pred = (y_pred_proba > best_threshold).astype(int))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Optimal Threshold (Max F1 on Validation): 0.9077
Max F1 Score at this threshold: 1.0000


In [100]:
print("\n--- Testing Pre-trained MLP on Dataset A (Source Domain) ---")

y_pred_proba_a = mlp_pretrain.predict(X_A_test).ravel()


# y_pred_a = (y_pred_proba_a > best_threshold).astype(int)

y_pred_a = (y_pred_proba_a > 0.5).astype(int)


# Calculate Metrics
accuracy_a = accuracy_score(y_A_test, y_pred_a) * 100
f1_a = f1_score(y_A_test, y_pred_a) * 100
roc_auc_a = roc_auc_score(y_A_test, y_pred_proba_a) * 100
recall_a = recall_score(y_A_test, y_pred_a) * 100
precision_a = precision_score(y_A_test, y_pred_a) * 100
specificity_a = recall_score(y_A_test, y_pred_a, pos_label=0) * 100

# Print the metrics
print(f"Pretraining Test Accuracy on Source domain: {accuracy_a:.2f}%")
print(f"Pretraining Test F1 on Source domain: {f1_a:.2f}%")
print(f"Pretraining Test Recall (Sensitivity) on Source domain: {recall_a:.2f}%")
print(f"Pretraining Test Specificity on Source domain: {specificity_a:.2f}%")
print(f"Pretraining Test ROC-AUC on Source domain: {roc_auc_score(y_pretrain_test, y_pred_proba_a):.2f}%")


--- Testing Pre-trained MLP on Dataset A (Source Domain) ---
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Pretraining Test Accuracy on Source domain: 83.87%
Pretraining Test F1 on Source domain: 86.84%
Pretraining Test Recall (Sensitivity) on Source domain: 89.19%
Pretraining Test Specificity on Source domain: 76.00%
Pretraining Test ROC-AUC on Source domain: 0.93%


In [101]:
print("\n--- Testing Pre-trained MLP on Dataset B (Zero-Shot Transfer) ---")

# Get raw probability predictions
y_pred_proba_b = mlp_pretrain.predict(X_B_test).ravel()

# Convert probabilities to binary class predictions
# y_pred_b = (y_pred_proba_b > best_threshold).astype(int)

y_pred_b = (y_pred_proba_b > 0.5).astype(int)


# Calculate Metrics
accuracy_b = accuracy_score(y_B_test, y_pred_b) * 100
f1_b = f1_score(y_B_test, y_pred_b) * 100
roc_auc_b = roc_auc_score(y_B_test, y_pred_proba_b) * 100
recall_b = recall_score(y_B_test, y_pred_b) * 100
precision_b = precision_score(y_B_test, y_pred_b) * 100
specificity_b = recall_score(y_B_test, y_pred_b, pos_label=0) * 100

print(f"Pretraining Test Accuracy on target domain: {accuracy_b:.2f}%")
print(f"Pretraining Test F1 on target domain: {f1_b:.2f}%")
print(f"Pretraining Test Recall (Sensitivity) on target domain: {recall_b:.2f}%")
print(f"Pretraining Test Specificity on target domain: {specificity_b:.2f}%")
print(f"Pretraining Test ROC-AUC on target domain: {roc_auc_score(y_finetune_test, y_pred_proba_b):.2f}%")


--- Testing Pre-trained MLP on Dataset B (Zero-Shot Transfer) ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Pretraining Test Accuracy on target domain: 77.42%
Pretraining Test F1 on target domain: 78.79%
Pretraining Test Recall (Sensitivity) on target domain: 92.86%
Pretraining Test Specificity on target domain: 64.71%
Pretraining Test ROC-AUC on target domain: 0.83%


### Finetuning on the target domain

In [102]:
def setup_fine_tuning(input_shape, pretrain_weights_path, best_params, fine_tune_lr=1e-1):
    # Create the model with exactly the same architecture as pre-training
    model_ft = create_mlp_model(input_shape, best_params)
    
    # Load pre-trained weights
    model_ft.load_weights(pretrain_weights_path)

    # Freeze layers for fine-tuning
    model_ft.get_layer('feature_layer_1').trainable = False
    model_ft.get_layer('feature_layer_2').trainable = True
    model_ft.get_layer('feature_layer_3').trainable = True
    
    # Recompile with lower learning rate for fine-tuning
    model_ft.compile(
        optimizer=Adam(learning_rate=fine_tune_lr),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc')]

    )
    
    finetune_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    return model_ft, finetune_scheduler


mlp_finetune, finetune_scheduler = setup_fine_tuning(
    input_shape=input_dim,  # Use the input dimension from Dataset B
    pretrain_weights_path='mlp_pretrain.weights.h5', 
    best_params=best_params,  # Add best_trial here
    fine_tune_lr=0.05
)



# Save the fine-tuned model weights (optional)
mlp_finetune.save_weights('mlp_finetune_weights.weights.h5')

# Check the model summary to ensure layers are frozen correctly
mlp_finetune.summary()


C:\Users\prosp\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_1 (Dense)              │ (None, 96)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_2 (Dense)              │ (None, 48)                  │           4,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_layer_3 (Dense)              │ (None, 24)                  │           1,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,913 (27.00 KB)

 Trainable params: 5,857 (22.88 KB)

 Non-trainable params: 1,056 (4.12 KB)

In [103]:
history_finetune = mlp_finetune.fit(
    # X_finetune_train_res, 
    # y_finetune_train_res,
    X_B_train, 
    y_B_train,
    epochs=300, # Fewer epochs are usually needed for fine-tuning
    batch_size=16, # Smaller batch size often works better for smaller datasets
    validation_data=(X_B_val, y_B_val),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True), 
    ],
    shuffle=False,
    verbose=1
)

print("Fine-tuning complete. The 'mlp_finetune' model is your final transfer model.")

Epoch 1/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7901 - auc: 0.8504 - loss: 0.5963 - val_accuracy: 0.6000 - val_auc: 1.0000 - val_loss: 0.4510
Epoch 2/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7443 - auc: 0.8408 - loss: 0.5430 - val_accuracy: 0.9333 - val_auc: 0.9955 - val_loss: 0.2813
Epoch 3/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8130 - auc: 0.8780 - loss: 0.4971 - val_accuracy: 0.9667 - val_auc: 0.9955 - val_loss: 0.2628
Epoch 4/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8359 - auc: 0.8979 - loss: 0.4474 - val_accuracy: 0.9000 - val_auc: 0.9821 - val_loss: 0.2467
Epoch 5/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8473 - auc: 0.9096 - loss: 0.4180 - val_accuracy: 0.9000 - val_auc: 0.9821 - val_loss: 0.2538
Epoch 6/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8244 - auc: 0.9013 - loss: 0.4466 - val_accuracy: 0.9000 - val_auc: 0.9821 - val_loss: 0.2624
Epoch 7/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9m

In [110]:
# --- 1. Use the fine-tune validation set (X_finetune_val) to find the optimal threshold ---
y_val_proba_ft = mlp_finetune.predict(X_B_val).ravel()  # Raw probability predictions

# Calculate precision, recall, and thresholds
precision, recall, thresholds = precision_recall_curve(y_B_val, y_val_proba_ft)

# Calculate F1 score for all thresholds
fscore = (2 * precision * recall) / (precision + recall + 1e-6)  # Adding 1e-6 to prevent division by zero

# Find the threshold that yields the maximum F1 score
ix = np.argmax(fscore)
best_finetune_threshold = thresholds[ix]

print(f"Optimal Threshold (Max F1 on Validation): {best_finetune_threshold:.4f}")
print(f"Max F1 Score at this threshold: {fscore[ix]:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Optimal Threshold (Max F1 on Validation): 0.3637
Max F1 Score at this threshold: 0.9655


In [111]:
# Get raw probability predictions
y_pred_proba_a_ft = mlp_finetune.predict(X_A_test).ravel()

# Convert probabilities to binary class predictions
y_pred_a_ft = (y_pred_proba_a_ft > best_finetune_threshold).astype(int)

# Calculate Metrics
accuracy_a_ft = accuracy_score(y_A_test, y_pred_a_ft) * 100
f1_a_ft = f1_score(y_A_test, y_pred_a_ft) * 100
roc_auc_a_ft = roc_auc_score(y_A_test, y_pred_proba_a_ft) * 100
recall_a_ft = recall_score(y_A_test, y_pred_a_ft) * 100
precision_a_ft = precision_score(y_A_test, y_pred_a_ft) * 100
specificity_a_ft = recall_score(y_A_test, y_pred_a_ft, pos_label=0) * 100

print(f"Finetune Test Accuracy on source domain: {accuracy_a_ft:.2f}%")
print(f"Finetune Test F1 on source domain: {f1_a_ft:.2f}%")
print(f"Finetune Test Recall (Sensitivity) on source domain: {recall_a_ft:.2f}%")
print(f"Finetune Test Specificity on source domain: {specificity_a_ft:.2f}%")
print(f"Finetune Test ROC-AUC on source domain: {roc_auc_score(y_pretrain_test, y_pred_proba_a_ft):.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Finetune Test Accuracy on source domain: 77.42%
Finetune Test F1 on source domain: 82.50%
Finetune Test Recall (Sensitivity) on source domain: 89.19%
Finetune Test Specificity on source domain: 60.00%
Finetune Test ROC-AUC on source domain: 0.84%


In [112]:
y_pred_proba_b_ft = mlp_finetune.predict(X_B_test).ravel()

# Convert probabilities to binary class predictions using a 0.5 threshold
y_pred_b_ft = (y_pred_proba_b_ft > best_finetune_threshold).astype(int)

# Calculate Metrics
accuracy_b_ft = accuracy_score(y_B_test, y_pred_b_ft) * 100
f1_b_ft = f1_score(y_B_test, y_pred_b_ft) * 100
roc_auc_b_ft = roc_auc_score(y_B_test, y_pred_proba_b_ft) * 100
recall_b_ft = recall_score(y_B_test, y_pred_b_ft) * 100
precision_b_ft = precision_score(y_B_test, y_pred_b_ft) * 100
specificity_b_ft = recall_score(y_B_test, y_pred_b_ft, pos_label=0) * 100


print(f"Finetune Test Accuracy on target domain: {accuracy_b_ft:.2f}%")
print(f"Finetune Test F1 on target domain: {f1_b_ft:.2f}%")
print(f"Finetune Test Recall (Sensitivity) on target domain: {recall_b_ft:.2f}%")
print(f"Finetune Test Specificity on target domain: {specificity_b_ft:.2f}%")
print(f"Finetune Test ROC-AUC on target domain:{roc_auc_score(y_finetune_test, y_pred_proba_b_ft):.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Finetune Test Accuracy on target domain: 83.87%
Finetune Test F1 on target domain: 83.87%
Finetune Test Recall (Sensitivity) on target domain: 92.86%
Finetune Test Specificity on target domain: 76.47%
Finetune Test ROC-AUC on target domain:0.93%


In [ ]:
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size' : '16'})

# Confusion matrix
cm_mlp = confusion_matrix(y_finetune_test, y_pred_b_ft)
# Unravel the confusion matrix to get TN, FP, FN, TP
tn, fp, fn, tp = cm_mlp.ravel()

plt.figure(figsize=(5, 4))
im = plt.imshow(cm_mlp, cmap='Blues', interpolation='nearest') 

vmax = cm_mlp.max()
colorbar_ticks = np.arange(0, vmax + 10, 10)
plt.colorbar(im, ticks=colorbar_ticks)

ax = plt.gca()

# Set X-axis labels (Predicted: Disease then No Disease)
plt.xticks([0, 1], ['Disease', 'No Disease'])
# Set Y-axis labels (Actual: Disease then No Disease)
plt.yticks([0, 1], ['No Disease', 'Disease'], rotation=90)  # Horizontal labels

plt.ylabel('Actual')
plt.xlabel('Predicted')

# Dynamic text color for contrast
norm = mcolors.Normalize(vmin=cm_mlp.min(), vmax=cm_mlp.max())
cmap = plt.colormaps['Blues']

for i in range(2):
    for j in range(2):
        cell_value = cm_mlp[i, j]
        cell_color = cmap(norm(cell_value))
        luminance = 0.299 * cell_color[0] + 0.587 * cell_color[1] + 0.114 * cell_color[2]
        text_color = 'white' if luminance < 0.5 else 'black'
        
        plt.text(j, i, f'{cell_value}', ha='center', va='center', color=text_color, fontsize=24, fontweight='bold')

# Save the confusion matrix image
plt.savefig('confusion_matrix_mlp.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
y_pred_prob_mlp = mlp_finetune.predict(X_finetune_test)
y_pred_prob_tabnet = tabnet_finetuned.predict(X_B_test)
y_pred_prob_xgb = xgb_finetune.predict_proba(X_finetune_test)[:, 1]


In [ ]:
plt.rcParams.update({'font.family': 'Times New Roman', 'font.weight': 'bold', 'font.size': 18})

# Get predicted probabilities for each model
y_pred_prob_mlp = mlp_finetune.predict(X_finetune_test)
y_pred_prob_tabnet = tabnet_finetuned.predict(X_B_test)  # Probabilities for the positive class (ROC)
y_pred_proba_xgb = xgb_finetune.predict(X_finetune_test) 

# Calculate ROC curve and AUC for each model
fpr_mlp, tpr_mlp, _ = roc_curve(y_finetune_test, y_pred_prob_mlp)
fpr_tabnet, tpr_tabnet, _ = roc_curve(y_B_test, y_pred_prob_tabnet)
fpr_xgb, tpr_xgb, _ = roc_curve(y_finetune_test, Dataset_B_Finetune)

# Calculate AUC for each model
roc_auc_mlp = auc(fpr_mlp, tpr_mlp)
roc_auc_tabnet = auc(fpr_tabnet, tpr_tabnet)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

# fpr, tpr, _ = roc_curve(y_finetune_test, Dataset_B_Finetune)

# Also calculate using roc_auc_score for verification
roc_auc_score_mlp = roc_auc_score(y_B_test, y_pred_prob_mlp)
roc_auc_score_tabnet = roc_auc_score(y_B_test, y_pred_prob_tabnet)
roc_auc_score_xgb = roc_auc_score(y_B_test, y_pred_proba_xgb)

print(f"MLP AUC (auc function): {roc_auc_mlp:.4f}")
print(f"MLP AUC (roc_auc_score): {roc_auc_score_mlp:.4f}")
print(f"TabNet AUC: {roc_auc_tabnet:.4f}")
print(f"XGBoost AUC: {roc_auc_xgb:.4f}")

# Plot combined ROC curve
plt.figure(figsize=(7, 6))

# Plot each ROC curve
plt.plot(fpr_mlp, tpr_mlp, color='red', lw=2, label=f'MLP (AUC = {roc_auc_mlp:.2f})')
plt.plot(fpr_tabnet, tpr_tabnet, color='green', lw=2, label=f'TabNet (AUC = {roc_auc_tabnet:.2f})')
plt.plot(fpr_xgb, tpr_xgb, color='orange', lw=2, label=f'XGBoost (AUC = {roc_auc_xgb:.2f})')

# Diagonal line for random classifier
plt.plot([0, 1], [0, 1], color='darkblue', lw=2, linestyle='--')

# Enable minor ticks for more grid lines
plt.minorticks_on()

# Customize grid settings for major and minor grid lines
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=1)

# Adjust plot settings
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Combined ROC Curve for Fine-Tuned Models')
plt.legend(loc='lower right')

# Add grid for better readability
plt.grid(True, alpha=0.4)


# Save the combined ROC curve
plt.savefig('roc_curve_combined.png', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()
plt.close()
